# Lecture 3 Overview

# Codewords

Starting simple: fixed-length bitstring
- reduce to n-bit substrings

# Hamming Distance

Distance between legal codewards
- measured in terms of number of bit flips

Efficient codes are of uniform Hamming Distance
- all codewords are equidistant from their neighbors

> Higher hamming distance = stronger error detection. More bits can be flipped and still detected. More complex errors can be detected. Bigger pool of errors that can be detected

## 2d + 1 Hamming Distance

Can **detect** up to 2d bit flips
- nxt codward is always 2d+1 bit flips away
- any fewer is guaranteed to land in the middle

Can **correct** up to d bit flips
- We just move to the closest codeword
- Unfortunately, no way to tell how many bit flips
  - E.g: 1, or (2d+1)-1?

```
                            [----  d = 1 ----]
C(000000) (000001) (000011) C(000111) (001111) (011111) C(111111)
                            [------------- 2d + 1 = 3 ----------]

```

> Note that detect > correct ALWAYS

# Encoding

We only send codewords
- Non-codewords indicate errors to receiver

But we want to **send any set of bitstrings**
- need to embed arbitrary input into sequence of codewords
- Adding **redundency**: `{TODO: fill in from slides}`

## Simple Embedding: **Parity**

Essentially Code with Hamming Distance 2
- can detect one bit flip (no correction capability)

Add extra bit to ensure odd(even) number of ones
- Code is 66% efficient (2 data bits for every 3 bits sent)

`{TODO: diagram/picture from slides}`

> Note: Even parity is simply XOR

## Simple Correction: **Voting**

Simply send each bit n times (e.g. n = 3)
- Code with Hamming distance 3 (d = 1)
- Can detect 2 bit flips
- Can correct 1 bit flip

Straightforward duplcation is extremely inefficient
- can be much smarter about this

`{TODO: diagram/picture from slides}`

# Per-Frame Detection Codes

> Per-Bit detection methods are very inefficient. More efficient to do things on the frame level

`[ --- Header --- | ------- Payload ------- | - EDC - ]`

More efficient: Add an error detection code per frame
- Frame is unit of transmission: all or nothing
- Computed over the entire frame -- INCLUDING THE HEADER!

Receiver checks EDC to make sure frame is valid
- if frame fails check, throw it away

We *could* use error-correcting codes
- But they are less efficient, and not useful if we **expect errors to be rare**
- Counter example: wireless comms

## Two-Dimentional Parity

```
  DATA    p
  0101001|1
D 1101001|0
A 1011110|1
T 0001110|1
A 0110100|1
  1011111|0
  -------/
p 1111011 0

```

Start w/ normal parity
- n data bits, 1 parity bit

Do the same across rows
- m data bits, 1 parity bit

Can detect up to 3 bit errors
- Even most 4 bit errors

Can correct any 1 bit error
- majority of (data) vs (row parity) vs (col parity) for correction

# Checksums

Simply sum up all of the data in the frame
- Transmit that sum as the EDC

Extremely lightweight
- Easy to compute fast in hardware
- Fragile: Hamming Distance of 2
  - So can only detect 1 bit errors

Also easy to modify if frame is modified in flight
- occurs often to packets on the internet

IP Packets include a `{TODO: fill in from slides}`

## IP 16b Checksum Example

1's complement of sum of words (not Bytes)
- Final 1's complement means all-zero frame is not valid

```
u_short cksum(u_short *buf, int count) {
    register u_long sum = 0;
    while (count--) {
        sum += *buf++;
        if (sum & 0xFFFF0000) {
            /* Carry occurred, so wrap around */
            sum &= 0xFFFF;
            sum++;
        }
    }
    return ~(sum & 0xFFFF);
}
```

`{TODO: rewatch this explanation in podcast for better understanding}`

## Checksum in Hardware

```
,-[ ]<-[ ]<-[ ]<-[ ]<-[ ]<-[ ]<-[ ]<-[ ]<-(+)<--
|                                          ^
`------------------------------------------`
```

Compute checksum in modulo-2 arithmetic
- Binary add/sub is simply XOR operation (equiv to vertical parity computation)

Need only a word-length shift register and XOR gate
- assuming data arrives serially
- all registers are intiially 0

`{TODO: rewatch the Checksum example in podcast as needed}`
- this demo makes sense now but it's easier to explain verbally than in text.

## From Sums to Remainders

Checksums are easy to compute but very FRAGILE
- In particular, **BURST** errors are frequently undetected
- We'd rathr have aschem that "smears" parity

Need to remain easy to implement in hardware
- so far, Shift reg + XOR gate 

We'll stick to Modulo-2 arithmetic
- Mult/Div are XOR based ass well


### Modulo-2 Mult/Div

Mult:
- shift, AND
- ADD the rows together

Div:
- shift, SUB, shift, SUB

#### Cyclic Remainder Check

Idea is to *divide* the incoming data, D, rather than add
- *divisor* is called **generator**, g

We can make CRC resilient to k-bit burst errors
- Need a generator of k+1 bits

Divide $2^kD$ by *g* to get remainder *r*
- Remainder is called **frame check sequence**

Send $2^kD-r$ (i.e., $2^kD XOR r)

`{TODO: copy rest of slides}`

`{TODO: rewatch podcast for explanation, it didn't quite stick this time}`